<a href="https://colab.research.google.com/github/fatima-bukhari5/liveness_detection_colab/blob/main/Hyper_Parameter_Tuning_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Library Imports


In [1]:
# Reference section to call important lib.

# mathemathical & data manupulation
import numpy as np          # numpy for mathemathical operations
import pandas as pd         # to manage data in two dimension structures

# O/S, command line, serialization, deserialization, iterations
import argparse             # command line arguments management.
import pickle               # pickle for object serialization/de-serialization
import os                   # to interact with operating system
import itertools            # to enhance iteration over object, i.e. for loop

# image processing
import cv2                  # opencv for image pre-processing
from imutils import paths   # another convienance for image processing like opencv

# For Data visualization & charts
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Reference to Scikit Learn for data pre-processing & post training
# performance evaluation. 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Reference to keras with tensorflow engine for model training.
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.models import Model
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
INIT_LR = 0.0001  # a variable for learning rate parameter while training
BS = 10           # batch size
EPOCHS = 16       # a variable for number of epochs parameter while training model

In [4]:
# location for training, validation & testing dataset 
train_path = '/content/drive/MyDrive/real_and_fake/train'
valid_path = '/content/drive/MyDrive/real_and_fake/valid'
test_path = '/content/drive/MyDrive/real_and_fake/test'

In [5]:
def read_image_data(X,Y, height, width):
    '''
    The following function is to convert the data into numpy array through
    `data_processing` function and resize it according vgg16 input size 
    of 224 by 224.
    
    Requirement:  Convert input images to heightxwidth for VGG16
    Usage:        Pre-Processing.
    Functionality:
                  1. load images one by one from folder, resize & attach lable.
                  2. Convert image to numpy array & return array with lable. 
    Parameters:
                  X => df['Image paths'] (dataframe column of image paths)
                  Y => df['Labels'] (dataframe column of image labels)

    Returns:      `image` in the form of numpy array & labels as encoded.
    '''
    # to do: variables & parmeters naming convention.
    # declaration Section
    data =[]                                      # An array to hold images.
    labels = Y                                    # A variable to hold label from parameter.
    # execution section    
    # to do: exception handling
    for path in X:                                # for each image in a given folder.
        image = cv2.imread(path)                  # reading image from given path.
        image = cv2.resize(image,(height,width))       # resizing image as of input requirement.
        data.append(image)                        # add image to declared array structure.
    data, labels = data_processing(data, labels)  # Calling & passing parameters to data_processing function.
    # to do: use python log api for console message, if any. print is childish.
    print('[INFO] Converting Data into Image form and encoding labels')
    # return section
    return data, labels

In [6]:
def data_organization(img_path):
    '''
    This function is to organize the data in the form of data frames to have clear understanding 
    and readability of data w.r.t to its labels

    Parameters: 
    img_path = folder path of images base folder as string

    Returns: dataframe with organized image paths and their labels
    '''
    print("Getting labels for images...")
    data = []
    labels = []
    list_image_path = []
    folders_list = os.listdir(img_path)
    
    # loop over all folders at a time. i.e real and fake
    for folder in folders_list:
        
        # extract the class label from the filename to add into the df
    
        folder_path = os.path.join(img_path,folder)
        images_list = os.listdir(folder_path)
        for image_name in images_list:
            label = folder
            imagePath = os.path.join(folder_path,image_name)
            labels.append(label)
            list_image_path.append(imagePath)
            
    image_path_label = {'image_path': list_image_path,'labels':labels}
    df_image_path_label = pd.DataFrame(image_path_label)
    
    print("DataFrame Complete....")
    return df_image_path_label

In [7]:
def data_processing(data,labels):
    '''
    Data processing to convert images data to numpy along with label encoding
    
    Parameters:
    data -> list of images data
    labels -> list of labels
    
    returns:
    data, labels as numpy array of image data and encoded labels
    '''
    data = np.array(data, dtype="float")
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    labels = to_categorical(labels, 2)
    
    return data, labels

In [8]:
def get_predictions(model, df_testing, height, width):
    '''
    Getting predictions w.r.t to the trained model as passed and the training data
    
    Parameters:
    model: trained model
    df_testing: data for testing in the form of dataframe
    
    Returns:
    
    Predictions as 0 or 1 and true_labels after data processing
    '''
    
    # getting test data in numpy 
    test_data, true_labels = read_image_data(df_testing['image_path'], df_testing['labels'], height, width) 
    
    # gettin predictions against the model
    print('[INFO] Getting Predictions')
    predictions = model.predict(test_data, steps=1, verbose=0)
    predictions_rounded = np.round(predictions)
    
    return predictions_rounded, true_labels

In [9]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [10]:
def model_performance(predictions, true_labels):
    '''
    The model displays a confusion matrix using a previously built confusion matrix function taken from sklearn.
    
    Parameters:
    predictions: previous predictions
    true_labels: array of actual labels to be compared
    
    Result: dislay confusion matrix
    '''
    print('[INFO] Getting Confusion Matrix')
    cm = confusion_matrix(true_labels[:,0], predictions[:,0])
    

    cm_plot_labels = ['real','fake']
    plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')
    

In [24]:
def build_model(num_classes, height, width, model_name):
    img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",)
    initializer = tf.keras.initializers.GlorotNormal(seed = None)
    inputs = layers.Input(shape=(height, width, 3,))
    x = img_augmentation(inputs)

    # importing the requires model
    if model_name == "effnet_v2l":
      model = keras.applications.efficientnet_v2.EfficientNetV2L(include_top=False, input_tensor=x, weights="imagenet")
    elif model_name == "effnet_b2":
      model = keras.applications.efficientnet.EfficientNetB7(include_top=False, input_tensor=x, weights="imagenet")
    elif model_name == "nas_net":
      model = keras.applications.nasnet.NASNetLarge(include_top=False, input_tensor=x, weights="imagenet")
    elif model_name == "inc_resnet_v2":
      model = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.Dense(3, kernel_initializer=initializer, activation='relu')
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(2, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name=model_name)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [25]:
def train_model_get_predictions():
  df_training = data_organization(train_path)
  df_validation = data_organization(valid_path)
  df_testing = data_organization(test_path)

  all_models = ['effnet_v2l', 'effnet_b2', 'nas_net', 'inc_resnet_v2']

  train_data, train_labels = read_image_data(df_training['image_path'], df_training['labels'],224, 224)
  val_data, val_labels = read_image_data(df_validation['image_path'], df_validation['labels'], 224, 224)

  all_predictions = {}
  for model_name in all_models:
    model = build_model(num_classes = 2, height=224, width=224, model_name=model_name)
    print("[INFO] Training model ", model_name)
    hist = model.fit(x=train_data, y=train_labels, validation_data=(val_data, val_labels), verbose=2, batch_size = BS,  
          validation_steps=5, steps_per_epoch=len(train_data) / BS, epochs=20)

    # get predictions
    predictions, true_labels = get_predictions(model = model, df_testing=df_testing, height = 224, width = 224)
    all_predictions[model_name] = {"predictions": predictions,
                                   "true_labels": true_labels, 
                                   "hist": hist}
  return all_predictions


In [26]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [27]:
all_results = train_model_get_predictions()

Getting labels for images...
DataFrame Complete....
Getting labels for images...
DataFrame Complete....
Getting labels for images...
DataFrame Complete....
[INFO] Converting Data into Image form and encoding labels
[INFO] Converting Data into Image form and encoding labels
[INFO] Training model  effnet_v2l
Epoch 1/20
30/30 - 221s - loss: 1.2264 - accuracy: 0.7100 - val_loss: 0.0365 - val_accuracy: 1.0000 - 221s/epoch - 7s/step
Epoch 2/20
30/30 - 190s - loss: 0.9493 - accuracy: 0.8133 - val_loss: 1.1211 - val_accuracy: 0.5200 - 190s/epoch - 6s/step
Epoch 3/20
30/30 - 189s - loss: 1.0957 - accuracy: 0.8167 - val_loss: 0.1440 - val_accuracy: 0.9200 - 189s/epoch - 6s/step
Epoch 4/20
30/30 - 193s - loss: 0.9914 - accuracy: 0.8133 - val_loss: 0.0345 - val_accuracy: 0.9800 - 193s/epoch - 6s/step
Epoch 5/20
30/30 - 193s - loss: 1.1856 - accuracy: 0.7967 - val_loss: 0.2697 - val_accuracy: 0.8800 - 193s/epoch - 6s/step
Epoch 6/20
30/30 - 196s - loss: 0.8237 - accuracy: 0.8267 - val_loss: 0.2434 

In [1]:
print("Effnet_v2l")
model_performance(predictions=all_results['effnet_v2l']['predictions'], true_labels = all_results['effnet_v2l']['true_labels'])

Effnet_v2l


NameError: ignored